## Overview


## Setup


For this lab, you will use the following libraries:

*   [`google-generativeai`](https://ai.google.dev/gemini-api/docs): Google's official Python SDK for the Gemini API.
*   [`langchain`](https://www.langchain.com/): Provides various chain and prompt functions from LangChain.
*   [`langchain-google-genai`](https://python.langchain.com/docs/integrations/chat/google_generative_ai/): Facilitates integration between LangChain and Google Gemini.


### Install required libraries

The following required libraries are __not__ preinstalled in the Skills Network Labs environment. __You must run the following cell__ to install them:

**Note:** The version has been pinned to ensure compatibility. It's recommended that you do the same. While future updates may be available, the pinned version ensures that the library continues to support this lab.

This might take approximately 1-2 minutes.

`%%capture` has been used to capture the installation, so you won’t see the process. However, once the installation is complete, a number will appear next to the cell.


In [2]:
%%capture
# Install Google Gemini SDK and LangChain integration
!pip install google-generativeai langchain-google-genai langchain langchain-core

After you install the libraries, restart your kernel:


In [3]:
import os

Once the kernel has been restarted, move on to the next part `Import required libraries`.


### Import required libraries


_It is recommended that you import all required libraries in one place (here):_


In [4]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# Google Gemini imports
import google.generativeai as genai

# LangChain + Google Gemini integration
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableSequence
from langchain_core.messages import HumanMessage, SystemMessage

### Set up the LLM


In this section, you will set up an LLM using **Google Gemini**. The following code initializes the Gemini model using your Google API key and wraps it into a function that allows for repeat use.


Some key parameters are explained here:
- `model` specifies which Gemini model you want to use. In this tutorial, you'll use `gemini-2.0-flash` (fast and free-tier friendly). Other options include `gemini-1.5-pro`, `gemini-2.0-flash-lite`, etc.
- `max_output_tokens` controls the maximum number of tokens in the generated output.
- `temperature` controls randomness/creativity (0.0 = deterministic, 1.0 = most creative).
- `top_p` (nucleus sampling) considers only the highest probability tokens.
- `top_k` limits token selection to the top k most likely tokens.
- You need a **Google API key** (free) from [Google AI Studio](https://aistudio.google.com/apikey).


## API Setup — Google Gemini (Free Tier)
This lab has been converted to use **Google Gemini** instead of IBM WatsonX. Google provides a **free tier** for the Gemini API which is more than enough for this lab.

### How to get your API key (takes 30 seconds):
1. Go to [Google AI Studio](https://aistudio.google.com/apikey)
2. Sign in with your Google account
3. Click **"Create API Key"**
4. Copy the key and paste it in the cell below

The free tier gives you **15 requests per minute** and **1,500 requests per day** with `gemini-2.0-flash` — plenty for learning!


##### Run the following cell to set up Google Gemini and create the `llm_model` function.


In [5]:
import os
from dotenv import load_dotenv

# ============================================================
# Remove corporate proxy settings that block Google API access
# (Walmart VPN/proxy intercepts HTTPS calls to external APIs)
# ============================================================
for proxy_var in ["HTTP_PROXY", "HTTPS_PROXY", "http_proxy", "https_proxy"]:
    os.environ.pop(proxy_var, None)

# ============================================================
# Load your Google Gemini API key from the .env file
# The .env file is in the same folder as this notebook
# with the line: GOOGLE_API_KEY=your_actual_key_here
# Get a free key from: https://aistudio.google.com/apikey
# ============================================================
load_dotenv()  # Reads the .env file and loads vars into os.environ
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if not GOOGLE_API_KEY or GOOGLE_API_KEY == "YOUR_API_KEY_HERE":
    raise ValueError("Please set your GOOGLE_API_KEY in the .env file!")

# Configure the Gemini SDK with your API key
genai.configure(api_key=GOOGLE_API_KEY)

def llm_model(prompt_txt, params=None):
    """
    Wrapper function that sends a prompt to Google Gemini and returns the response.
    
    Parameters (optional dict):
        - max_output_tokens: Max tokens in the response (default: 256)
        - temperature: Controls randomness 0.0-1.0 (default: 0.5)
        - top_p: Nucleus sampling threshold (default: 0.2)
        - top_k: Limits token selection (default: 1)
    """
    # Default generation parameters
    default_params = {
        "max_output_tokens": 256,
        "temperature": 0.5,
        "top_p": 0.2,
        "top_k": 1
    }

    # If user passes custom params, merge them
    if params:
        # Map WatsonX-style param names to Gemini equivalents for convenience
        mapped = {}
        for k, v in params.items():
            if k == "max_new_tokens":
                mapped["max_output_tokens"] = v
            elif k == "min_new_tokens":
                continue  # Gemini doesn't have this param, skip it
            else:
                mapped[k] = v
        default_params.update(mapped)

    # Create the Gemini model with the generation config
    model = genai.GenerativeModel(
        model_name="gemini-2.0-flash",
        generation_config=genai.types.GenerationConfig(**default_params)
    )
    
    # Generate a response
    response = model.generate_content(prompt_txt)
    return response.text

# Quick test to verify the API key works
print(llm_model("Say hello in one sentence."))

Hello there, I hope you're having a great day!



##### (Optional) Test with a longer prompt to verify everything works


In [6]:
# Test with the same prompt from the original notebook
print(llm_model("Hey my name is Swapnil Dhiman who is an iOS developer working in Walmart with 3.5 years of experience. Seeing the rise of AI in the current field, the best detailed course of action for me would be, so that my job is secure in tech industry."))


Okay, Swapnil, that's a smart and proactive question. With 3.5 years of iOS development experience at Walmart, you have a solid foundation. Here's a detailed course of action to leverage AI and secure your future in the tech industry, broken down into steps:

**I. Understanding the Landscape: AI's Impact on iOS Development**

*   **Research Current AI Applications in iOS:**
    *   **Machine Learning Frameworks:**  Familiarize yourself with Core ML (Apple's machine learning framework), TensorFlow Lite (for mobile), and potentially PyTorch Mobile.  Understand how these frameworks are used to integrate pre-trained models into iOS apps.
    *   **AI-Powered Features in iOS:**  Analyze how Apple is already using AI in iOS:
        *   **Siri:**  Natural language processing, speech recognition.
        *   **Photos App:**  Object recognition, face detection, scene understanding.
        *   **Camera:**  Computational photography, image enhancement.
        *   **QuickType Keyboard:**  Predi

Let's see the available Gemini models and the parameters we can configure.


In [7]:
# List available Gemini models
print("Available Gemini models:\n")
for model in genai.list_models():
    if "generateContent" in model.supported_generation_methods:
        print(f"  - {model.name}")

print("\n\nCommon generation parameters for Gemini:")
print("""
  max_output_tokens : Max tokens in the response (e.g., 256, 512, 1024)
  temperature       : Randomness 0.0 (deterministic) to 2.0 (very creative), default 1.0
  top_p             : Nucleus sampling 0.0-1.0, default 0.95
  top_k             : Top-k sampling (integer), default 40
  stop_sequences    : List of strings that stop generation when encountered
""")

Available Gemini models:

  - models/gemini-2.5-flash
  - models/gemini-2.5-pro
  - models/gemini-2.0-flash
  - models/gemini-2.0-flash-001
  - models/gemini-2.0-flash-exp-image-generation
  - models/gemini-2.0-flash-lite-001
  - models/gemini-2.0-flash-lite
  - models/gemini-exp-1206
  - models/gemini-2.5-flash-preview-tts
  - models/gemini-2.5-pro-preview-tts
  - models/gemma-3-1b-it
  - models/gemma-3-4b-it
  - models/gemma-3-12b-it
  - models/gemma-3-27b-it
  - models/gemma-3n-e4b-it
  - models/gemma-3n-e2b-it
  - models/gemini-flash-latest
  - models/gemini-flash-lite-latest
  - models/gemini-pro-latest
  - models/gemini-2.5-flash-lite
  - models/gemini-2.5-flash-image
  - models/gemini-2.5-flash-preview-09-2025
  - models/gemini-2.5-flash-lite-preview-09-2025
  - models/gemini-3-pro-preview
  - models/gemini-3-flash-preview
  - models/gemini-3-pro-image-preview
  - models/nano-banana-pro-preview
  - models/gemini-robotics-er-1.5-preview
  - models/gemini-2.5-computer-use-preview-

## Prompt engineering

[Prompt engineering](https://www.ibm.com/think/topics/prompt-engineering?utm_source=skills_network&utm_content=in_lab_content_link&utm_id=Lab-In-Context+Learning+and+Prompt+Templates-v3-GenAIcourse_1741386184) is the art and science of crafting effective inputs for large language models to generate desired outputs. As language models have evolved in capability and size, so too has the importance of how we communicate with them. Prompt engineering involves strategically designing text prompts that guide an AI model's responses toward specific goals, formats, or reasoning patterns.

**[In-context learning](https://research.ibm.com/blog/demystifying-in-context-learning-in-large-language-model?utm_source=skills_network&utm_content=in_lab_content_link&utm_id=Lab-In-Context+Learning+and+Prompt+Templates-v3-GenAIcourse_1741386184)** represents one of the most fascinating capabilities of modern large language models. It is a model's ability to "learn" from examples provided directly within the prompt itself, without any updates to its underlying parameters or weights. This capability allows models to adapt to new tasks or domains simply by demonstrating what success looks like through examples.

By combining the principles of prompt engineering with the power of in-context learning, developers can guide language models to perform a remarkably diverse range of tasks with unprecedented flexibility and efficiency. Let's now explore these methods in detail.


### Basic prompt

A **basic prompt** is the simplest form of prompting, where you provide a short text or phrase to the model without any special formatting or instructions. The model generates a continuation based on patterns it has learned during training. Basic prompts are useful for exploring the model's capabilities and understanding how it naturally responds to minimal input.


In this example, let's introduce a basic prompt that uses specific parameters to guide the language model's response. You'll then define a simple prompt and retrieve the model's response.

The prompt used is "The wind is." Let the model generate itself.


In [8]:
params = {
    "max_new_tokens": 128,
    "min_new_tokens": 10,
    "temperature": 0.5,
    "top_p": 0.2,
    "top_k": 1
}

prompt = "The wind is "

# Getting a reponse from the model with the provided prompt and new parameters
response = llm_model(prompt, params)
print(f"prompt: {prompt}\n")
print(f"response : {response}\n")

prompt: The wind is 

response : The wind is **blowing**.




As you can see from the response, the model continues generating content based on the initial prompt, "The wind is." You might notice that the response appears truncated or incomplete. This is because you have set the `max_new_tokens,` which restricts the number of tokens the model can generate.

Try adjusting the parameters and observe how the response changes.


### Exercise 1
Experiment with different basic prompts by changing the input phrase. Try these prompts and compare the different responses:
1. "The future of artificial intelligence is"
2. "Once upon a time in a distant galaxy"
3. "The benefits of sustainable energy include"


In [9]:
## Your code here

<details>
    <summary>Click here for hints</summary>

```python
params = {
    "max_new_tokens": 128, # Try 256 or 512 for more detailed answers
    "min_new_tokens": 10, # Increase to 25-50 if you want more substantial answers
    "temperature": 0.5, # Controls randomness in generation (0.0-1.0)
                       # Lower (0.1-0.3): More focused, consistent, factual responses
                      # Higher (0.7-1.0): More creative, diverse, unpredictable outputs
    "top_p": 0.2, # Nucleus sampling - considers only highest probability tokens
                       # Lower values (0.1-0.3): More conservative, focused text
                       # Higher values (0.7-0.9): More diverse vocabulary and ideas
    "top_k": 1 # Limits token selection to top k most likely tokens
                       # 1 = greedy decoding (always picks most likely token)
                       # Try 40-50 for more varied outputs
}

# Compare responses to different prompts
prompts = [
    "The future of artificial intelligence is",
    "Once upon a time in a distant galaxy",
    "The benefits of sustainable energy include"
]

for prompt in prompts:
    response = llm_model(prompt, params)
    print(f"prompt: {prompt}\n")
    print(f"response : {response}\n")
```
</details>


### Zero-shot prompt

[**Zero-shot prompting**](https://www.ibm.com/think/topics/zero-shot-prompting?utm_source=skills_network&utm_content=in_lab_content_link&utm_id=Lab-In-Context+Learning+and+Prompt+Templates-v3-GenAIcourse_1741386184) is a technique where the model performs a task without any examples or prior specific training on that task. This approach tests the model's ability to understand instructions and apply its knowledge to a new context without demonstration. Zero-shot prompts typically include clear instructions about what the model should do, allowing it to leverage its pre-trained knowledge effectively.


---
Here is an example of a zero-shot prompt:

Zero-shot learning is crucial for testing a model's ability to apply its pre-trained knowledge to new, unseen tasks without additional training. This capability is valuable for gauging the model's generalization skills.

In this example, let's demonstrate a zero-shot learning scenario using a prompt that asks the model to classify a statement without any prior specific training on similar tasks. The prompt requests the model to assess the truthfulness of the statement: "The Eiffel Tower is located in Berlin." After defining the prompt, you'll execute it with default parameters and print the response.

This approach helps you understand how well the model can handle direct questions based on its underlying knowledge and reasoning abilities.


Try running the prompt to see the model's capacity to correctly analyze and respond to factual inaccuracies.


In [10]:
prompt = """Classify the following statement as true or false: 
            'The Eiffel Tower is located in Berlin.'

            Answer:
"""
response = llm_model(prompt, params)
print(f"prompt: {prompt}\n")
print(f"response : {response}\n")

prompt: Classify the following statement as true or false: 
            'The Eiffel Tower is located in Berlin.'

            Answer:


response : False




The model responds with the 'False' answer, which is correct. It also gives the reason for the answer.


### Exercise 2
Create zero-shot prompts for the following tasks:
1. Write a prompt that asks the model to classify a movie review as positive or negative.
2. Create a prompt that instructs the model to summarize a paragraph about climate change.
3. Design a prompt that asks the model to translate an English phrase to Spanish without showing any examples.


In [11]:
## Starter code: provide your solutions in the TODO parts

# 1. Prompt for Movie Review Classification
movie_review_prompt = #TODO

# 2. Prompt for Climate Change Paragraph Summarization
climate_change_prompt = ## TODO

# 3. Prompt for English to Spanish Translation
translation_prompt = ## TODO

responses = {}
responses["movie_review"] = ## TODO
responses["climate_change"] = ## TODO
responses["translation"] = ## TODO

for prompt_type, response in responses.items():
    print(f"=== {prompt_type.upper()} RESPONSE ===")
    print(response)
    print()

SyntaxError: invalid syntax (1152506969.py, line 4)

<details>
    <summary>Click here for hints</summary>

```python
# 1. Prompt for Movie Review Classification
movie_review_prompt = """
Classify the following movie review as either 'positive' or 'negative'.

Review: "I was extremely disappointed by this film. The plot was predictable, the acting was wooden, and the special effects looked cheap. I can't recommend this to anyone."

Classification:
"""

# 2. Prompt for Climate Change Paragraph Summarization
climate_change_prompt = """
Summarize the following paragraph about climate change in no more than two sentences.

Paragraph: "Climate change refers to long-term shifts in temperatures and weather patterns. These shifts may be natural, but since the 1800s, human activities have been the main driver of climate change, primarily due to the burning of fossil fuels like coal, oil and gas, which produces heat-trapping gases. The consequences of climate change include more frequent and severe droughts, storms, and heat waves, rising sea levels, melting glaciers, and warming oceans which can directly impact biodiversity, agriculture, and human health."

Summary:
"""

# 3. Prompt for English to Spanish Translation
translation_prompt = """
Translate the following English phrase into Spanish.

English: "I would like to order a coffee with milk and two sugars, please."

Spanish:
"""

responses = {}
responses["movie_review"] = llm_model(movie_review_prompt)
responses["climate_change"] = llm_model(climate_change_prompt)
responses["translation"] = llm_model(translation_prompt)

for prompt_type, response in responses.items():
    print(f"=== {prompt_type.upper()} RESPONSE ===")
    print(response)
    print()
```
</details>


### One-shot prompt

[**One-shot prompting**](https://www.ibm.com/think/topics/one-shot-prompting?utm_source=skills_network&utm_content=in_lab_content_link&utm_id=Lab-In-Context+Learning+and+Prompt+Templates-v3-GenAIcourse_1741386184) provides the model with a **single** example of the task before asking it to perform a similar task. This technique gives the model a pattern to follow, improving its understanding of the desired output format and style. One-shot learning is particularly useful when you want to guide the model's response without extensive examples.


Here is a one-shot learning example where the model is given a single example to help guide its translation from English to French.

The prompt provides a sample translation pairing, "How is the weather today?" translated to "Comment est le temps aujourd'hui?" This example serves as a guide for the model to understand the task context and desired format. The model is then tasked with translating a new sentence, "Where is the nearest supermarket?" without further guidance.


In [ ]:
params = {
    "max_new_tokens": 20,
    "temperature": 0.1,
}

prompt = """Here is an example of translating a sentence from English to French:

            English: “How is the weather today?”
            French: “Comment est le temps aujourd'hui?”
            
            Now, translate the following sentence from English to French:
            
            English: “Where is the nearest supermarket?”
            
"""
response = llm_model(prompt, params)
print(f"prompt: {prompt}\n")
print(f"response : {response}\n")

prompt: Here is an example of translating a sentence from English to French:

            English: “How is the weather today?”
            French: “Comment est le temps aujourd'hui?”

            Now, translate the following sentence from English to French:

            English: “Where is the nearest supermarket?”



response : French: "Où est le supermarché le plus proche ?"




The model's response shows how it applies the structure and context provided by the initial example to translate the new sentence.


Consider experimenting with different sentences or adjusting the parameters to see how these changes impact the model's translations.


### Exercise 3
Develop one-shot prompts for these scenarios:
1. Create a prompt with one example of a formal email, then ask the model to write another formal email on a different topic.
2. Provide one example of converting a technical concept into a simple explanation, then ask the model to explain a different concept.
3. Give one example of extracting keywords from a sentence, then ask the model to extract keywords from a new sentence.


In [ ]:
## Starter code: provide your solutions in the TODO parts

# 1. One-shot prompt for formal email writing
formal_email_prompt = ## TODO

# 2. One-shot prompt for simplifying technical concepts
technical_concept_prompt = ## TODO

# 3. One-shot prompt for keyword extraction
keyword_extraction_prompt = ## TODO

responses = {}
responses["formal_email"] = ## TODO
responses["technical_concept"] = ## TODO
responses["keyword_extraction"] = ## TODO

for prompt_type, response in responses.items():
    print(f"=== {prompt_type.upper()} RESPONSE ===")
    print(response)
    print()

SyntaxError: invalid syntax (3502624225.py, line 4)

<details>
    <summary>Click here for hints</summary>

```python
# 1. One-shot prompt for formal email writing
formal_email_prompt = """
Here is an example of a formal email requesting information:

Subject: Inquiry Regarding Product Specifications for Model XYZ-100

Dear Customer Support Team,

I hope this email finds you well. I am writing to request detailed specifications for your product Model XYZ-100. Specifically, I am interested in learning about its dimensions, power requirements, and compatibility with third-party accessories.

Could you please provide this information at your earliest convenience? Additionally, I would appreciate any available documentation or user manuals that you could share.

Thank you for your assistance in this matter.

Sincerely,
John Smith

---

Now, please write a formal email to a university admissions office requesting information about their application deadline and required documents for the Master's program in Computer Science:

"""

# 2. One-shot prompt for simplifying technical concepts
technical_concept_prompt = """
Here is an example of explaining a technical concept in simple terms:

Technical Concept: Blockchain
Simple Explanation: A blockchain is like a digital notebook that many people have copies of. When someone writes a new entry in this notebook, everyone's copy gets updated. Once something is written, it can't be erased or changed, and everyone can see who wrote what. This makes it useful for recording important information that needs to be secure and trusted by everyone.

---

Now, please explain the following technical concept in simple terms:

Technical Concept: Machine Learning
Simple Explanation:
"""

# 3. One-shot prompt for keyword extraction
keyword_extraction_prompt = """
Here is an example of extracting keywords from a sentence:

Sentence: "Cloud computing offers businesses flexibility, scalability, and cost-efficiency for their IT infrastructure needs."
Keywords: cloud computing, flexibility, scalability, cost-efficiency, IT infrastructure

---

Now, please extract the main keywords from the following sentence:

Sentence: "Sustainable agriculture practices focus on biodiversity, soil health, water conservation, and reducing chemical inputs."
Keywords:
"""

responses = {}
responses["formal_email"] = llm_model(formal_email_prompt)
responses["technical_concept"] = llm_model(technical_concept_prompt)
responses["keyword_extraction"] = llm_model(keyword_extraction_prompt)

for prompt_type, response in responses.items():
    print(f"=== {prompt_type.upper()} RESPONSE ===")
    print(response)
    print()
```
</details>


### Few-shot prompt

[**Few-shot prompting**](https://www.ibm.com/think/topics/few-shot-prompting?utm_source=skills_network&utm_content=in_lab_content_link&utm_id=Lab-In-Context+Learning+and+Prompt+Templates-v3-GenAIcourse_1741386184) extends the one-shot approach by providing multiple examples (typically 2-5) before asking the model to perform the task. These examples establish a clearer pattern and context, helping the model better understand the expected output format, style, and reasoning. This technique is particularly effective for complex tasks where a single example might not convey all the nuances.


Here is an example of few-shot learning by classifying emotions from text statements. 

Let's provide the model with three examples, each labeled with an appropriate emotion—joy, frustration, and sadness—to establish a pattern or guideline on how to categorize emotions in statements.

After presenting these examples, let's challenge the model with a new statement: "That movie was so scary I had to cover my eyes." The task for the model is to classify the emotion expressed in this new statement based on the learning from the provided examples. 


In [12]:
#parameters: Set `max_new_tokens` to 10, which constrains the model to generate brief responses

params = {
    "max_new_tokens": 10,
}

prompt = """Here are few examples of classifying emotions in statements:

            Statement: 'I just won my first marathon!'
            Emotion: Joy
            
            Statement: 'I can't believe I lost my keys again.'
            Emotion: Frustration
            
            Statement: 'My best friend is moving to another country.'
            Emotion: Sadness
            
            Now, classify the emotion in the following statement:
            Statement: 'That movie was so scary I had to cover my eyes.’
            

"""
response = llm_model(prompt, params)
print(f"prompt: {prompt}\n")
print(f"response : {response}\n")

prompt: Here are few examples of classifying emotions in statements:

            Statement: 'I just won my first marathon!'
            Emotion: Joy

            Statement: 'I can't believe I lost my keys again.'
            Emotion: Frustration

            Statement: 'My best friend is moving to another country.'
            Emotion: Sadness

            Now, classify the emotion in the following statement:
            Statement: 'That movie was so scary I had to cover my eyes.’




response : Emotion: Fear




The parameters are set with `max_new_tokens` to 10, which constrains the model to generate brief responses, focusing on the essential output without elaboration.


The model's response demonstrates its ability to use the provided few examples to understand and classify the emotion of the new statement effectively following the same pattern in examples.


### Chain-of-thought (CoT) prompt

[**Chain-of-thought (CoT) prompting**](https://www.ibm.com/think/topics/chain-of-thoughts?utm_source=skills_network&utm_content=in_lab_content_link&utm_id=Lab-In-Context+Learning+and+Prompt+Templates-v3-GenAIcourse_1741386184) encourages the model to break down complex problems into step-by-step reasoning before arriving at a final answer. By explicitly showing or requesting intermediate steps, this technique improves the model's problem-solving abilities and reduces errors in tasks requiring multi-step reasoning. CoT is particularly effective for mathematical problems, logical reasoning, and complex decision-making tasks.


Here is an example of the CoT prompting technique, designed to guide the model through a sequence of reasoning steps to solve a problem. In this example, the problem is a simple arithmetic question: “A store had 22 apples. They sold 15 apples today and received a new delivery of 8 apples. How many apples are there now?”

The CoT technique involves structuring the prompt by instructing the model to “Break down each step of your calculation.” This encourages the model to include explicit reasoning steps, mimicking human-like problem-solving processes.


In [13]:
params = {
    "max_new_tokens": 512,
    "temperature": 0.5,
}

prompt = """Consider the problem: 'A store had 22 apples. They sold 15 apples today and got a new delivery of 8 apples. 
            How many apples are there now?’

            Break down each step of your calculation

"""
response = llm_model(prompt, params)
print(f"prompt: {prompt}\n")
print(f"response : {response}\n")

prompt: Consider the problem: 'A store had 22 apples. They sold 15 apples today and got a new delivery of 8 apples. 
            How many apples are there now?’

            Break down each step of your calculation



response : Here's the breakdown of the calculation:

1. **Start:** The store begins with 22 apples.

2. **Sales:** They sell 15 apples, so we subtract that from the initial amount: 22 - 15 = 7 apples.

3. **Delivery:** They receive a delivery of 8 apples, so we add that to the remaining amount: 7 + 8 = 15 apples.

**Answer:** There are now 15 apples in the store.




From the response of the model, you can see the prompt directs the model to:

1. Add the initial number of apples to the apples received in the new delivery.
2. Subtract the number of apples sold from the sum obtained in the first step.


By breaking down the problem into specific steps, the model is better able to understand the sequence of operations required to arrive at the correct answer.


### Exercise 4
Create CoT prompts for these scenarios:
1. Write a prompt that asks the model to think through whether a student should study tonight or go to a movie with friends, considering their upcoming test in two days.
2. Write a prompt that instructs the model to explain the step-by-step process of making a peanut butter and jelly sandwich.


In [14]:
## Starter code: provide your solutions in the TODO parts

# 1. Prompt for decision-making process
decision_making_prompt = ## TODO

# 2. Prompt for explaining a process
sandwich_making_prompt = ## TODO

responses = {}
responses["decision_making"] = ## TODO
responses["sandwich_making"] = ## TODO

for prompt_type, response in responses.items():
    print(f"=== {prompt_type.upper()} RESPONSE ===")
    print(response)
    print()

SyntaxError: invalid syntax (2065588509.py, line 4)

<details>
    <summary>Click here for hints</summary>

```python
# 1. Prompt for decision-making process
decision_making_prompt = """
Consider this situation: A student is trying to decide whether to study tonight or go to a movie with friends. They have a test in two days.

Think through this decision step-by-step, considering the pros and cons of each option, and what factors might be most important in making this choice.
"""

# 2. Prompt for explaining a process
sandwich_making_prompt = """
Explain how to make a peanut butter and jelly sandwich.

Break down each step of the process in detail, from gathering ingredients to finishing the sandwich.
"""

responses = {}
responses["decision_making"] = llm_model(decision_making_prompt)
responses["sandwich_making"] = llm_model(sandwich_making_prompt)

for prompt_type, response in responses.items():
    print(f"=== {prompt_type.upper()} RESPONSE ===")
    print(response)
    print()
```
</details>


### Self-consistency

[**Self-consistency**](https://www.promptingguide.ai/techniques/consistency) is an advanced technique in which the model generates multiple independent solutions or answers to the same problem, then evaluates these different approaches to determine the most consistent or reliable result. This method enhances accuracy by leveraging the model's ability to approach problems from different angles and identify the most robust solution through comparison and verification.


This example demonstrates the self-consistency technique by reasoning through multiple calculations for a single problem. The problem posed is: “When I was 6, my sister was half my age. Now I am 70, what age is my sister?”

The prompt instructs, “Provide three independent calculations and explanations, then determine the most consistent result.” This encourages the model to engage in critical thinking and consistency checking, both of which are vital for complex decision-making processes.


In [15]:
params = {
    "max_new_tokens": 512,
}

prompt = """When I was 6, my sister was half of my age. Now I am 70, what age is my sister?

            Provide three independent calculations and explanations, then determine the most consistent result.

"""
response = llm_model(prompt, params)
print(f"prompt: {prompt}\n")
print(f"response : {response}\n")

prompt: When I was 6, my sister was half of my age. Now I am 70, what age is my sister?

            Provide three independent calculations and explanations, then determine the most consistent result.



response : Here are three independent calculations to determine your sister's age, along with explanations, and a final determination:

**Calculation 1: Age Difference**

*   **Explanation:** The age difference between you and your sister remains constant throughout your lives.
*   **Calculation:**
    *   When you were 6, your sister was 6 / 2 = 3 years old.
    *   The age difference is 6 - 3 = 3 years.
    *   Therefore, your sister is always 3 years younger than you.
    *   Now that you are 70, your sister is 70 - 3 = 67 years old.

**Calculation 2: Relative Age**

*   **Explanation:** While the *ratio* of your ages changes, the *difference* remains constant. We can use the initial ratio to establish the difference and apply it to your current age.
*   **Calculation:**
    *   At 

The model's response demonstrates three different calculations and explanations, each using a distinct logical approach to determine the sister's age.

Self-consistency can help identify the most accurate and reliable answer in scenarios where multiple plausible solutions exist.


## Applications of prompting in different use cases


In this section, we'll demonstrate how to leverage LangChain's prompt templates to build practical applications with consistent, reproducible results. Each application follows a common pattern using the LCEL approach:

1. Define the content or problem to be addressed.
2. Create a template with variables for dynamic content.
3. Convert the template into a LangChain PromptTemplate.
4. Build a chain using the pipe operator `|` to connect:

    - Input variables
    - The prompt template
    - The LLM
    - An output parser


5. Invoke the chain with specific inputs to generate results.

This structured approach enables you to create reusable components for various NLP tasks while maintaining flexibility to adjust parameters and inputs. You'll see how this pattern applies across different use cases.


### Introduction to LangChain 

[LangChain](https://www.langchain.com/) is a powerful framework designed to simplify the development of applications powered by language models. Built to address the challenges of working with LLMs in practical settings, LangChain provides a standardized interface for connecting models with various data sources and application environments.

LangChain serves as an abstraction layer, making it easier to build complex LLM applications without handling the low-level details of model interaction. This framework has become a standard tool in the LLM ecosystem, supporting a wide range of use cases from chatbots to document analysis systems.

In this section, we'll focus on LangChain's prompt template capabilities, demonstrating how they can be used to create structured, reproducible interactions with language models across different application types."


### Prompt template


[Prompt templates](https://python.langchain.com/v0.2/docs/concepts/#prompt-templates) are a key concept in LangChain. They help translate user input and parameters into instructions for a language model. These templates can be used to guide a model's response, helping it understand the context and generate relevant and coherent language-based outputs.

A prompt template acts as a reusable structure for generating prompts with dynamic values. It allows you to define a consistent format while leaving placeholders for variables that change with each use case. This approach makes prompting more systematic and maintainable, especially when working with complex applications.

**Modern LangChain (as of 2025) offers two main approaches to working with templates:**

- The traditional `LLMChain` approach
- The newer LangChain Expression Language (LCEL) pattern using the pipe operator `|` for more flexible composition

LCEL has become the recommended pattern for building LangChain applications as it offers better composability, clearer visualization of data flow, and more flexibility when constructing complex chains.

**To use a prompt template with LCEL, you typically follow these steps:**

- Define your template with variables in curly braces `{}`
- Create a `PromptTemplate` instance
- Build a chain using the pipe operator `|` to connect components
- Invoke the chain with your input values

Let's initialize a LangChain-compatible LLM using **Google Gemini**, then demonstrate this approach. We will use the `gemini-2.0-flash` model:


In [16]:
# Initialize Gemini as a LangChain-compatible LLM
# ChatGoogleGenerativeAI wraps the Gemini API for use with LangChain chains
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",               # The Gemini model to use
    google_api_key=GOOGLE_API_KEY,           # Your API key (set earlier)
    max_output_tokens=256,                   # Controls max tokens in the generated output
    temperature=0.5,                         # Controls randomness/creativity of responses
)
llm

ChatGoogleGenerativeAI(profile={'max_input_tokens': 1048576, 'max_output_tokens': 8192, 'image_inputs': True, 'audio_inputs': True, 'pdf_inputs': True, 'video_inputs': True, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'image_tool_message': True, 'tool_choice': True}, google_api_key=SecretStr('**********'), model='gemini-2.0-flash', temperature=0.5, max_output_tokens=256, client=<google.genai.client.Client object at 0x117bc1090>, default_metadata=(), model_kwargs={})

Use the `PromptTemplate` to create a template for a string-based prompt. In this template, you'll define two parameters: `adjective` and `content`. These parameters allow for the reuse of the prompt across different situations. For instance, to adapt the prompt to various contexts, simply pass the relevant values to these parameters.


In [17]:
template = """Tell me a {adjective} joke about {content}.
"""
prompt = PromptTemplate.from_template(template)
prompt 

PromptTemplate(input_variables=['adjective', 'content'], input_types={}, partial_variables={}, template='Tell me a {adjective} joke about {content}.\n')

Now, let's take a look at how the prompt has been formatted.


In [18]:
prompt.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.\n'

From the response, you can see that the prompt is formatted according to the specified context.


To ensure consistent formatting of the prompts, we will define a helper function `format_prompt`. This function takes a dictionary of variables and applies them to our prompt template. It ensures that all placeholder variables (like {adjective} and {content}) are properly replaced with their values before the prompt is sent to the language model.


In [19]:
from langchain_core.runnables import RunnableLambda

# Define a function to ensure proper formatting
# Since ChatGoogleGenerativeAI is a chat model, we wrap the formatted string
# into a HumanMessage so the model receives it correctly.
def format_prompt(variables):
    return [HumanMessage(content=prompt.format(**variables))]

In [20]:
#When used in a function call, ** "unpacks" a dictionary into separate keyword arguments. 
#Example:
#python
def greet(name, age):
    print(f"Hello {name}, you are {age}!")

data = {"name": "Alice", "age": 25}
greet(**data)  # Equivalent to greet(name="Alice", age=25)

Hello Alice, you are 25!


The following code builds a chain using the LCEL (LangChain Expression Language) pattern. This chain connects components using the pipe operator (`|`) to create a processing flow. The chain takes input variables, passes them through the prompt template, sends the formatted prompt to the LLM, and uses a string output parser to return the final response.


In [21]:
print("HTTP_PROXY:", os.environ.get("HTTP_PROXY", "not set"))
print("HTTPS_PROXY:", os.environ.get("HTTPS_PROXY", "not set"))
print("http_proxy:", os.environ.get("http_proxy", "not set"))
print("https_proxy:", os.environ.get("https_proxy", "not set"))
print("NO_PROXY:", os.environ.get("NO_PROXY", "not set"))

HTTP_PROXY: not set
HTTPS_PROXY: not set
http_proxy: not set
https_proxy: not set
NO_PROXY: 127.0.0.1,localhost,.walmart.net,.prod.walmart.com,.qa.walmart.com,.cloud.walmart.com,.homeoffice.wal-mart.com,.cld.samsclub.com,.walmartlabs.com,.wmt,.local,.bfd.walmart.com,.gecwalmart.com,.walmart.com,wmlink


In [22]:
# Remove proxy settings that block Google API access
for var in ["HTTP_PROXY", "HTTPS_PROXY", "http_proxy", "https_proxy"]:
    os.environ.pop(var, None)

In [23]:
import urllib.request
try:
    resp = urllib.request.urlopen("https://generativelanguage.googleapis.com", timeout=5)
    print("Connection OK:", resp.status)
except Exception as e:
    print("Cannot reach Google API:", e)

Cannot reach Google API: HTTP Error 404: Not Found


In [24]:
# Create the chain with explicit formatting
joke_chain = (
    RunnableLambda(format_prompt)
    | llm 
    | StrOutputParser()
)

# Run the chain
response = joke_chain.invoke({"adjective": "funny", "content": "chickens"})
print(response)

Why did the chicken cross the playground?

To get to the other slide!


From the response, you can see the LLM came up with a funny joke about chickens.

To use this prompt in another context, simply replace the variables accordingly.


In [25]:
response = joke_chain.invoke({"adjective": "sad", "content": "fish"})
print(response)

Why did the fisherman cry when he caught a fish?

...Because he realized he was taking it away from its plaice.


In the following sections, you will learn how to create agents capable of completing various tasks using prompt templates.


### Text summarization Agent



Here is a text summarization agent designed to help summarize the content you provide to the LLM. The LCEL chain takes your content as input, processes it through the prompt template, sends it to the language model, and returns a concise summary.

You can store the content to be summarized in a variable, allowing for repeated use with different texts:


In [28]:
content = """
    The rapid advancement of technology in the 21st century has transformed various industries, including healthcare, education, and transportation. 
    Innovations such as artificial intelligence, machine learning, and the Internet of Things have revolutionized how we approach everyday tasks and complex problems. 
    For instance, AI-powered diagnostic tools are improving the accuracy and speed of medical diagnoses, while smart transportation systems are making cities more efficient and reducing traffic congestion. 
    Moreover, online learning platforms are making education more accessible to people around the world, breaking down geographical and financial barriers. 
    These technological developments are not only enhancing productivity but also contributing to a more interconnected and informed society.
"""
name = """
Swapnil Dhiman
"""
location = """
Bangalore, India
"""
template = f"""Summarize the {content} in one sentence. The name of the user is {name} and the location is {location}
"""
prompt = PromptTemplate.from_template(template)

# Create the LCEL chain
summarize_chain = (
    RunnableLambda(format_prompt)
    | llm 
    | StrOutputParser()
)

# Run the chain
summary = summarize_chain.invoke({"content": content, "name": name, "location": location})
print(summary)

Swapnil Dhiman from Bangalore, India, notes that rapid technological advancements like AI, IoT, and online learning are revolutionizing industries, enhancing productivity, and creating a more interconnected and informed society.


### Question answering


Here is a Q&A agent built using the LCEL pattern.

This agent enables the LLM to learn from the provided content and answer questions based on what it has learned. Occasionally, if the LLM does not have sufficient information, it may generate a speculative answer. To manage this, we'll specifically instruct it to respond with "Unsure about the answer" if it is uncertain about the correct response.

The chain takes both the content (context) and question as inputs, processing them through our template before sending them to the LLM:


In [ ]:
content = """
    The solar system consists of the Sun, eight planets, their moons, dwarf planets, and smaller objects like asteroids and comets. 
    The inner planets—Mercury, Venus, Earth, and Mars—are rocky and solid. 
    The outer planets—Jupiter, Saturn, Uranus, and Neptune—are much larger and gaseous.
"""

question = "Which planets in the solar system are rocky and solid?"

template = """
    Answer the {question} based on the {content}.
    Respond "Unsure about answer" if not sure about the answer.
    
    Answer:
    
"""
prompt = PromptTemplate.from_template(template)

# Create the LCEL chain
qa_chain = (
    RunnableLambda(format_prompt)
    | llm 
    | StrOutputParser()
)

# Run the chain
answer = qa_chain.invoke({"question": question, "content": content})
print(answer)

### Text classification


Here is a text classification agent designed to categorize text into predefined categories. This example employs zero-shot learning, where the agent classifies text without prior exposure to related examples.

Using the LCEL approach, we create a chain that takes both the text to be classified and the available categories as inputs:


In [ ]:
text = """
    The concert last night was an exhilarating experience with outstanding performances by all artists.
"""

categories = "Entertainment, Food and Dining, Technology, Literature, Music."

template = """
    Classify the {text} into one of the {categories}.
    
    Category:
    
"""
prompt = PromptTemplate.from_template(template)

# Create the LCEL chain
classification_chain = (
    RunnableLambda(format_prompt)
    | llm 
    | StrOutputParser()
)

# Run the chain
category = classification_chain.invoke({"text": text, "categories": categories})
print(category)

### Code generation


Here is an example of an SQL code generation agent built with LCEL. This agent is designed to generate SQL queries based on provided descriptions. It interprets the requirements from your input and translates them into executable SQL code.

The chain takes your natural language description and transforms it into a properly formatted SQL query:


In [ ]:
description = """
    Retrieve the names and email addresses of all customers from the 'customers' table who have made a purchase in the last 30 days. 
    The table 'purchases' contains a column 'purchase_date'
"""

template = """
    Generate an SQL query based on the {description}
    
    SQL Query:
    
"""
prompt = PromptTemplate.from_template(template)

# Create the LCEL chain
sql_generation_chain = (
    RunnableLambda(format_prompt) 
    | llm 
    | StrOutputParser()
)

# Run the chain
sql_query = sql_generation_chain.invoke({"description": description})
print(sql_query)

### Role playing


You can also configure the LLM to assume specific roles as defined by us, enabling it to follow predetermined rules and behave like a task-oriented chatbot.

This approach separates the role definition from the prompt structure, allowing for easy role-switching without rewriting the entire prompt. The key components are:

- `role`: Specifies the character, expertise, or persona the LLM should embody
- `tone`: Defines the communication style and emotional quality of responses
- `question`: Contains the user's query that needs addressing

By parameterizing these elements, you can rapidly change the LLM's behavior by adjusting single variables rather than rewriting entire prompts. This pattern is particularly valuable for building conversational agents that need to serve different functions or adapt to various contexts.

For example, the code below configures the LLM to act as a game master. In this role, the LLM answers questions about games while maintaining an engaging and immersive tone, enhancing the user experience. You can test the bot by asking questions related to tabletop role-playing games or game mastering. Try asking about game rules, storytelling techniques, player management, or setting descriptions such as:

1. "Who are you?"
2. "What are the basic rules of Dungeons & Dragons?"
3. "How do I create a balanced encounter for my players?"
4. "Can you describe a mysterious forest setting for my adventure?"
5. "What's a good puzzle I could use in my dungeon?"
6. "How do I handle a player who is constantly interrupting others?"

The function is written within a while loop, allowing continuous interaction. **To exit the loop and terminate the conversation, type "quit," "exit," or "bye" into the input box.**


In [ ]:
role = """
    Dungeon & Dragons game master
"""

tone = "engaging and immersive"

template = """
    You are an expert {role}. I have this question {question}. I would like our conversation to be {tone}.
    
    Answer:
    
"""
prompt = PromptTemplate.from_template(template)

# Create the LCEL chain
roleplay_chain = (
    RunnableLambda(format_prompt)
    | llm 
    | StrOutputParser()
)

# Helper function — call this with your question instead of using input()
# (input() doesn't work reliably in Cursor/Jupyter notebooks)
def ask_game_master(question):
    response = roleplay_chain.invoke({"role": role, "question": question, "tone": tone})
    print("Answer:", response)

# Try it out! Change the question and re-run this cell.
ask_game_master("Who are you?")
# ask_game_master("What are the basic rules of Dungeons & Dragons?")
# ask_game_master("Can you describe a mysterious forest setting for my adventure?")

Answer:  Alright, adventurer! Welcome, welcome! Settle in by the virtual fire, grab a tankard of something refreshing (or perhaps a healing potion?), and tell me, what grand quest is stirring in your heart today? What mysteries beckon you from the shadowed corners of the world?

I am here to guide you, to challenge you, and to weave a tapestry of adventure that will leave you breathless. Don't be shy! Tell me everything. What question burns within you? What problem needs solving? What treasure are you seeking?

The fate of kingdoms, the balance of the cosmos, the simple joy of a well-earned victory... all possibilities lie before us. So, speak! Let the adventure begin! *cracks knuckles and leans forward expectantly*


### Exercise 5

**Create an LCEL Chain with Custom Formatting**

In this exercise, you'll create your own LCEL chain that uses prompt templates to build a custom application.

**Task:** Create a product review analyzer that can:
1. Identify the sentiment (positive, negative, or neutral).
2. Extract mentioned product features.
3. Provide a one-sentence summary of the review.

**Steps:**
1. Create a prompt template with placeholders for the review text.
2. Build an LCEL chain that formats your prompt properly.
3. Process the sample reviews and display the results.
4. Try modifying the chain to change the output format.

**Sample input:**
```python
reviews = [
    "I love this smartphone! The camera quality is exceptional and the battery lasts all day. The only downside is that it heats up a bit during gaming.",
    "This laptop is terrible. It's slow, crashes frequently, and the keyboard stopped working after just two months. Customer service was unhelpful."
]


In [ ]:
## Starter code: provide your solutions in the TODO parts
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser

# Initialize Gemini LLM for this exercise (with more tokens for detailed analysis)
# TODO: Initialize your LLM
llm = ## TODO  # Hint: use ChatGoogleGenerativeAI with max_output_tokens=512 and temperature=0.2

# Here is an example template you can use
template = """
Analyze the following product review:
"{review}"

Provide your analysis in the following format:
- Sentiment: (positive, negative, or neutral)
- Key Features Mentioned: (list the product features mentioned)
- Summary: (one-sentence summary)
"""

# TODO: Create your prompt template
product_review_prompt = ## TODO

# TODO: Create a formatting function
def format_review_prompt(variables):
    # Your code here — remember to wrap in HumanMessage for chat models!
    pass

# TODO: Build your LCEL chain
review_analysis_chain = (
    # Your code here
)

# Example reviews to process
reviews = [
    "I love this smartphone! The camera quality is exceptional and the battery lasts all day. The only downside is that it heats up a bit during gaming.",
    "This laptop is terrible. It's slow, crashes frequently, and the keyboard stopped working after just two months. Customer service was unhelpful."
]

# TODO: Process the reviews
for review in reviews:
    # Your code here
    pass

<details>
    <summary>Click here for hints</summary>

```python
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage

# Initialize Gemini LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=GOOGLE_API_KEY,
    max_output_tokens=512,
    temperature=0.2,
)

# Create the prompt template
template = """
Analyze the following product review:
"{review}"

Provide your analysis in the following format:
- Sentiment: (positive, negative, or neutral)
- Key Features Mentioned: (list the product features mentioned)
- Summary: (one-sentence summary)
"""

product_review_prompt = PromptTemplate.from_template(template)

# Create a formatting function (wrap in HumanMessage for chat model)
def format_review_prompt(variables):
    return [HumanMessage(content=product_review_prompt.format(**variables))]

# Build the LCEL chain
review_analysis_chain = (
    RunnableLambda(format_review_prompt)
    | llm 
    | StrOutputParser()
)

# Process the reviews
reviews = [
    "I love this smartphone! The camera quality is exceptional and the battery lasts all day. The only downside is that it heats up a bit during gaming.",
    "This laptop is terrible. It's slow, crashes frequently, and the keyboard stopped working after just two months. Customer service was unhelpful."
]

for i, review in enumerate(reviews):
    print(f"==== Review #{i+1} ====")
    result = review_analysis_chain.invoke({"review": review})
    print(result)
    print()
```
</details>


## Conclusion

Congratulations on completing this lab on prompt engineering and LangChain prompt templates! You've successfully navigated from basic prompting techniques to more advanced approaches including zero-shot, one-shot, few-shot learning, as well as chain-of-thought reasoning and self-consistency prompting. You then applied these concepts practically using LangChain's prompt templates with the modern LCEL pattern to create various applications.

By learning how to properly structure prompts and build composable chains with the pipe operator (`|`), you've gained essential skills for developing robust LLM applications. These techniques provide a solid foundation for creating more complex systems and getting the most out of any language model you work with.


## Authors


[Hailey Quach](https://www.haileyq.com) is a Data Scientist at IBM.

[Kang Wang](https://author.skills.network/instructors/kang_wang) is a Data Scientist at IBM. He is also a PhD Candidate in the University of Waterloo.

[Faranak Heidari](https://author.skills.network/instructors/faranak_heidari) is a Data Scientist at IBM. 


<!---
## Change log

|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2025-03-07|1.1|Hailey Quach|Updated lab|
|2025-04-03|1.2|Jojy John|ID Reviewed|
|2025-04-03|1.2|Rahul Rawat|QA pass|
--->


© Copyright IBM Corporation. All rights reserved.
